In [1]:
import numpy as np
import pandas as pd
import re
import itertools
from collections import Counter
from tensorflow.contrib import learn
import pickle
from sklearn.model_selection import KFold
from BasicTextCNN import BasicTextCNN
from PositionTextCNN import PositionTextCNN

import tensorflow as tf
import numpy as np
import os
import time
import datetime
from tensorflow.contrib import learn
import pandas as pd

In [2]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


def load_data_and_labels():
    x_text = sentence_support_df.tokenizedSentenceFromPaper.as_matrix()
    y = sentence_support_df.label.as_matrix()
    y = [[0, 1] if x == 1 else [1, 0] for x in y  ]
    return [x_text, np.array(y)]

def compute_pathway_name_terms(pathway):
    pathway = pathway.replace('signaling', '').replace('pathway', '').replace('-', ' ')
    return [t for t in pathway.lower().strip().split() if len(t)>1]

def tokenize_pathway_names(sentence, pathwayA, pathwayB):
    genesA = [gene.lower() for gene in pathway_to_genes_dict[pathwayA]] + compute_pathway_name_terms(pathwayA)
    genesB = [gene.lower() for gene in pathway_to_genes_dict[pathwayB]] + compute_pathway_name_terms(pathwayB)
    tokenized_sentence = []
    for word in sentence.lower().split():
        token = None
        for gene in genesA:
            if gene in word:
                token = 'pathwayA'
                break
                
        for gene in genesB:
            if gene in word:
                token = 'pathwayB'
                break
        if token is None:
            token = word
        tokenized_sentence.append(token)
    return ' '.join(tokenized_sentence)

def compute_distance_embedding(word, x):
    word_distances = np.zeros(x.shape, dtype='int')
    for i in range(x.shape[0]):
        word_positions = np.where(x[i] == word)[0]
        for j in range(x.shape[1]):
            if len(word_positions) > 0:
                word_position = word_positions[np.argmin(np.abs(word_positions - j))]
                word_distances[i][j] = word_position - j
                if word_distances[i][j]<0:
                    word_distances[i][j] = 600+word_distances[i][j]
            else:
                word_distances[i][j] = 299
    return word_distances

In [3]:
pathway_to_genes_dict = pickle.load(open( "data/pathway_to_genes_dict.p", "rb" ))
sentence_support_df = pd.read_csv('data/sentence_support_v3.tsv', delimiter='\t')
sentence_support_df.drop_duplicates(inplace=True)
sentence_support_df['tokenizedSentenceFromPaper'] = sentence_support_df.apply(lambda x: tokenize_pathway_names(x.sentenceFromPaper, x.pathwayA, x.pathwayB), axis=1)

In [4]:
# Load data
print("Loading data...")
x_text, y = load_data_and_labels()

Loading data...


In [5]:
# Build vocabulary
max_document_length = max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))
print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))

Vocabulary Size: 33447


In [6]:
encodedPathwayA, encodedPathwayB = list(vocab_processor.transform(['pathwayA pathwayB']))[0][:2]
encodedPathwayA, encodedPathwayB

(8, 53)

In [7]:
word_distancesA = compute_distance_embedding(encodedPathwayA, x)
word_distancesB = compute_distance_embedding(encodedPathwayB, x)    

In [8]:
# Creating folds
kf = KFold(n_splits=4, random_state=5, shuffle=True)
for k, (train_index, test_index) in enumerate(kf.split(x, y)):
# for train_index, test_index in kf.split(x):
#     print("Fold: %s =>" % k,  "TRAIN:", train_index, "TEST:", test_index)
    x_train, x_dev = x[train_index], x[test_index]
    y_train, y_dev = y[train_index], y[test_index]
    
    train_word_distancesA = word_distancesA[train_index]
    train_word_distancesB = word_distancesB[train_index]
    
    test_word_distancesA = word_distancesA[test_index]
    test_word_distancesB = word_distancesB[test_index]
    
    print("Fold: %s =>" % k, "Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
    
    
    model = PositionTextCNN(sequence_length=x_train.shape[1],
            vocab_processor=vocab_processor, num_epochs=1, evaluate_every=300, results_dir='fold%s'%k)
    model.train_network(x_train, y_train, x_dev, y_dev, 
                        train_word_distancesA, train_word_distancesB, test_word_distancesA, test_word_distancesB)
    break

Fold: 0 => Train/Dev split: 31795/10599
INFO:tensorflow:Summary name word_embedding/W:0/grad/hist is illegal; using word_embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name word_embedding/W:0/grad/sparsity is illegal; using word_embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:

KeyboardInterrupt: 

In [9]:
sess = tf.Session()
one_hot_encoding = tf.one_hot(list(range(8)), 8)
sess.run(one_hot_encoding)

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]], dtype=float32)